In [1]:
#execute related notebooks within the scope of the current one to use their variables without the need for individual imports or file creation
#%run dev_pressure_curves.ipynb dev_machine_learning.ipynb

In [2]:
#Here, we import pressure curve files/object to run through a simple workflow. %store -[OPTION] var to -r : retrieve, -d : delete, -z : clear all  
%store -r pc 

In [3]:
methods_plot = pc.plot_methods()
methods_plot.show()

methods = pc.get_methods()
print("Methods: ", methods)

Methods:  ['SAA_411_Irino.M', 'SAA_411_5FU.M', 'SAA_411_Doc.M', 'SAA_411_Gem.M', 'SAA_411_Pac.M']


In [4]:
indices = pc.get_method_indices('SAA_411_Pac.M')
pac_plots = pc.plot_batches(indices)
pac_plots.show()

In [5]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative

processor = PressureCurvesMethodExtractFeaturesNative(period=10, bins=6, window_size=7)
pc = processor.run(pc)

In [6]:
batches = [batch for batch in pc.get_batches() if "Pac" in batch]
print("Batches: ", batches)

batches.sort()
print("Sorted by date: ", batches)

tests = batches[2:]

Batches:  ['210813_Pac 2021-08-13 10-37-27', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54', '210819_Pac 2021-08-19 15-50-49', '210812_Pac 2021-08-12 10-30-07', '210826_Pac 2021-08-26 16-15-14']
Sorted by date:  ['210812_Pac 2021-08-12 10-30-07', '210813_Pac 2021-08-13 10-37-27', '210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [7]:
print("test batch dates: ", tests) # used to get train data, then discarded and test sets iteratively selected by increasing batch date

test batch dates:  ['210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [8]:
# create training set
first_test_group = tests[0]
print("First test group date: ", first_test_group)

first_test = first_test_group.split(" ")[-2:]
first_test = " ".join(first_test)
first_test = first_test.replace(":", "-")

date_threshold_min = first_test

First test group date:  210819_Pac 2021-08-19 15-50-49


In [9]:
train_indices = []
for i in indices:
    meta = pc.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = start_time.replace(":", "-")
    else:
        start_time = start_time.strftime("%Y-%m-%d %H-%M-%S")
    if batch_position > 5 and start_time < date_threshold_min:
        train_indices.append(i)

train_data = pc.get_features(train_indices)
train_metadata = pc.get_metadata(train_indices)
train_data.to_csv("dev/workflow_train_features.csv", index=False) # test data files will include batch date
train_metadata.to_csv("dev/workflow_train_metadata.csv", index=False)

train_size = len(train_indices)
print("Number of training curves: ", train_size)

Number of training curves:  18


In [10]:
fig_train=pc.plot_pressure_curves(train_indices)
fig_train.update_layout(showlegend=False)
fig_train.show()

In [11]:
fig_train_features = pc.plot_features(train_indices)
fig_train_features.show()

In [12]:
train_data.describe() # maybe this can be fed to model to guide feature selection for splits based on importance. Increase weight of less-impactful statistical features like area

,runtime,area_0.0_0.66,min_0.0_0.66,max_0.0_0.66,mean_0.0_0.66,std_0.0_0.66,range_0.0_0.66,residual_std_0.0_0.66,relative_change_0.0_0.66,area_0.665_1.325,...,residual_std_2.665_3.325,relative_change_2.665_3.325,area_3.33_3.975,min_3.33_3.975,max_3.33_3.975,mean_3.33_3.975,std_3.33_3.975,range_3.33_3.975,residual_std_3.33_3.975,relative_change_3.33_3.975
count,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,239.7,45.615564,59.168333,70.964444,69.082222,2.263282,11.796111,0.290950,0.191190,45.607461,...,0.161157,0.086774,29.953785,40.111667,53.816667,46.444145,3.986602,13.705000,0.179988,0.336882
std,0.0,0.138899,0.438718,0.245146,0.210181,0.097076,0.442641,0.028393,0.009539,0.118714,...,0.013107,0.006906,0.060148,0.208616,0.273646,0.093785,0.017303,0.251027,0.021201,0.007567
min,239.7,45.382050,58.140000,70.470000,68.727368,2.127925,11.170000,0.226699,0.177064,45.395150,...,0.138974,0.075803,29.886325,39.760000,53.390000,46.339769,3.956200,13.200000,0.140243,0.324318
25%,239.7,45.535100,58.835000,70.840000,68.959511,2.195601,11.500000,0.275235,0.184337,45.535106,...,0.153399,0.080869,29.913681,39.907500,53.630000,46.380019,3.972435,13.510000,0.166760,0.333250
50%,239.7,45.597825,59.340000,70.940000,69.057180,2.238455,11.680000,0.298989,0.190649,45.599787,...,0.161608,0.087999,29.930850,40.120000,53.745000,46.409308,3.983898,13.750000,0.180211,0.334498
75%,239.7,45.703975,59.470000,71.020000,69.216090,2.329823,12.042500,0.307609,0.196815,45.658975,...,0.174386,0.092370,29.983619,40.250000,53.977500,46.492596,4.003031,13.857500,0.193109,0.342449
max,239.7,45.881100,59.780000,71.510000,69.484060,2.465887,12.720000,0.328231,0.213450,45.869025,...,0.178920,0.096431,30.103775,40.430000,54.360000,46.677385,4.011977,14.110000,0.225321,0.350559


In [13]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

ml_ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ml_ana)

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html




MachineLearningAnalyses 
  variables: 18 rows, 49 columns
  metadata: 18 rows, 14 columns



In [14]:
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scaler = MachineLearningScaleFeaturesScalerSklearn(scaler_type = "StandardScaler")
ml_ana = scaler.run(ml_ana)

fig_train_features = ml_ana.plot_data()
fig_train_features.update_layout(title="Train set features")
fig_train_features.show()

In [15]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iforest = MachineLearningMethodIsolationForestSklearn()

ml_ana = iforest.run(ml_ana)

ml_ana.train()
fig_train_scores = ml_ana.plot_scores()
fig_train_scores.show()

In [16]:
# new tests sorted by date
tests = {}
for batch in batches:
    batch_date = batch.split(" ")[-2 : ]
    batch_date = " ".join(batch_date)

    if batch_date >= date_threshold_min: # search and collect test samples
        date_threshold_min = batch_date 
        
        test_indices = pc.get_batch_indices(batch)
        test_size = len(test_indices)

        for i in range(test_size):
            tests[f"{batch_date}_{i+1}"] = test_indices[i]
            
    else:
        continue

In [17]:
dates = list(tests.keys())
print(dates)

['2021-08-19 15-50-49_1', '2021-08-19 15-50-49_2', '2021-08-19 15-50-49_3', '2021-08-19 15-50-49_4', '2021-08-19 15-50-49_5', '2021-08-19 15-50-49_6', '2021-08-19 15-50-49_7', '2021-08-19 15-50-49_8', '2021-08-19 15-50-49_9', '2021-08-19 15-50-49_10', '2021-08-19 15-50-49_11', '2021-08-19 15-50-49_12', '2021-08-19 15-50-49_13', '2021-08-19 15-50-49_14', '2021-08-19 15-50-49_15', '2021-08-19 15-50-49_16', '2021-08-19 15-50-49_17', '2021-08-26 16-15-14_1', '2021-08-26 16-15-14_2', '2021-08-26 16-15-14_3', '2021-08-26 16-15-14_4', '2021-08-26 16-15-14_5', '2021-08-26 16-15-14_6', '2021-08-26 16-15-14_7', '2021-08-26 16-15-14_8', '2021-08-26 16-15-14_9', '2021-08-26 16-15-14_10', '2021-08-26 16-15-14_11', '2021-08-26 16-15-14_12', '2021-08-26 16-15-14_13', '2021-08-26 16-15-14_14', '2021-08-26 16-15-14_15', '2021-08-26 16-15-14_16', '2021-08-26 16-15-14_17', '2021-08-26 16-15-14_18', '2021-08-26 16-15-14_19', '2021-08-26 16-15-14_20', '2021-09-06 11-09-22_1', '2021-09-06 11-09-22_2', '2021

In [18]:
print("Number of test samples: ", len(dates))

Number of test samples:  65


In [43]:
n = 3
date = dates[n]

test_data = pc.get_features(tests[date])
test_metadata = pc.get_metadata(tests[date])

print(f"Test {date}: index:", tests[date])
train_indices.append(tests[date])

fig_test_features = pc.plot_features(train_indices, normalize=True)
for trace in fig_test_features.data[:-1]:
    trace.line.color = "black"
fig_test_features.show()

fig_test_features_raw = pc.plot_features_raw(train_indices)
fig_test_features_raw.update_layout(showlegend=False)
fig_test_features_raw.show()

Test 2021-08-19 15-50-49_4: index: 119


In [44]:
ml_ana.predict(test_data, test_metadata)

outliers = ml_ana.test_prediction_outliers() # defaults: n_tests = _get_num_tests(), show_scores = False
print(outliers)

#fig_test_curves = pc.plot_pressure_curves(test_metadata["index"].tolist())
fig_test_curves = pc.plot_pressure_curves(train_indices)
train_indices.remove(tests[date])
fig_test_curves.update_layout(showlegend=False)
for trace in fig_test_curves.data[:-1]:
    trace.line.color = "black"
fig_test_curves.show()

fig_test_scores = ml_ana.plot_scores()
fig_test_scores.update_layout(title=f"Test set {date} final run")
fig_test_scores.show()

Setting true_classes to majority class...
   index  batch_position  train_size  train_time  threshold     score  \
0    119               4          20    0.162968  -0.085024 -0.080228   

   confidence   class  
0        0.94  normal  


In [45]:
confidence_plot = ml_ana.plot_confidence_variation()
confidence_plot.show()

In [46]:
import os

test_data.drop(columns="runtime", inplace=True)
data = pd.concat([test_metadata, test_data], axis=1)

if outliers["class"].iloc[0] == "outlier":
    outlier_data = data

    if os.path.exists("dev/outliers.csv"):
        outliers_file = pd.read_csv("dev/outliers.csv")
        outliers_file = pd.concat([outliers_file, outlier_data], axis = 0)
        outliers_file.drop_duplicates(subset="index", inplace = True, ignore_index=True)
        outliers_file.to_csv("dev/outliers.csv", index=False)
    
    else:
        outlier_data.to_csv("dev/outliers.csv", index=False)

else:

    if os.path.exists("dev/normals.csv"):
        normals_file = pd.read_csv("dev/normals.csv")

        normals_file = pd.concat([normals_file, data], axis = 0)
        normals_file.drop_duplicates(subset="index", inplace=True, ignore_index=True)
        normals_file.to_csv("dev/normals.csv", index=False)
    
    else:
        data.to_csv("dev/normals.csv", index=False)

ml_ana.add_prediction() # add_data already calls self.train()

In [47]:
threshold_plot = ml_ana.plot_threshold_variation()
threshold_plot.show()
threshold_plot.write_image("dev/figures/fig_threshold_variation_serialized_tests.png", width=1100, height= 350, scale = 3)

In [48]:
train_time_plot = ml_ana.plot_train_time()
train_time_plot.show()
train_time_plot.write_image("dev/figures/fig_train_time_variation.png", width=1100, height= 350, scale = 3)

In [49]:
stability_plot = ml_ana.plot_model_stability()
stability_plot.show()
stability_plot.write_image("dev/figures/fig_model_stability_vs_train_size.png", width=1100, height= 350, scale = 3)

In [50]:
ml_ana.get_results(summarize = True)

Index: 116
Train size: 18
Num. Tests: 38
Normal: 0
Outlier: 38

Index: 117
Train size: 18
Num. Tests: 38
Normal: 35
Outlier: 3

Index: 118
Train size: 19
Num. Tests: 38
Normal: 37
Outlier: 1

Index: 119
Train size: 20
Num. Tests: 37
Normal: 23
Outlier: 14


